In [1]:
import numpy as np
import pandas as pd
import pickle
import math
import os
import matplotlib.pyplot as plt
import collections
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
###################
n_f=7+365*7
n_node=365
###################

###node ddata -single
if False:#True:#
    N=12121
    np.random.seed(1)
    indices = np.random.permutation(N);print('indices',indices[0])
    n_training=int(N*0.6)+4
    n_val=int(N*0.2)-2
    n_test=N-n_training-n_val
    training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]
    
    nf1=np.load('data/nf.npy')#n,7
    x1=np.load('data/disp.npy')#n,365,7
    print('nf1.shape',nf1.shape,'x1.shape',x1.shape)
    
    '''#scaling
    for i in range(7):
        m=np.mean(nf1[...,i].flatten());std=np.std(nf1[...,i].flatten())
        nf1[...,i]=(nf1[...,i]-m)/std
    for i in range(7):
        m=np.mean(x1[...,i].flatten());std=np.std(x1[...,i].flatten())
        x1[...,i]=(x1[...,i]-m)/std'''
    
    #input6
    input6=np.concatenate((x1[:,-6:,:].reshape((-1,6*7)),nf1),axis=-1)
    print('input6.shape,',input6.shape,)
    #label359
    label359=x1[:,:-6,:].reshape((-1,359*7))
    print('label359.shape,',label359.shape,)
    #input 365
    features=np.concatenate((x1.reshape((-1,365*7)),nf1),axis=1)
    print('features.shape',features.shape)
    
    #x - split
    f_training, f_val, f_test = features[training_idx],features[val_idx], features[test_idx]
    input6_training, input6_val, input6_test = input6[training_idx],input6[val_idx], input6[test_idx]
    label359_training, label359_val, label359_test = label359[training_idx],label359[val_idx], label359[test_idx]
    print(f_training.shape,f_val.shape,f_test.shape)
    
    
    #x - save
    save_flag=True#False#
    if save_flag:
        data_path='data/data_mlp/single/input365/'
        np.savez_compressed(data_path+'features_train',f_training)
        np.savez_compressed(data_path+'features_val',f_val)
        np.savez_compressed(data_path+'features_test',f_test)
        data_path='data/data_mlp/single/input6/'
        np.savez_compressed(data_path+'features_train',input6_training)
        np.savez_compressed(data_path+'features_val',input6_val)
        np.savez_compressed(data_path+'features_test',input6_test)
        data_path='data/data_mlp/single/label359/'
        np.savez_compressed(data_path+'label_train',label359_training)
        np.savez_compressed(data_path+'label_val',label359_val)
        np.savez_compressed(data_path+'label_test',label359_test)




###node ddata -multi -. mulst be revised for imputation data -> complete
if False:#True:#
    N=12121+75000
    #split idx
    np.random.seed(1)
    indices = np.random.permutation(N);print('indices',indices[0])
    n_training=int(N*0.6)
    n_val=int(N*0.2)
    n_test=N-n_training-n_val
    training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]
    
    
    nf1=np.load('data/nf.npy')
    x1=np.load('data/disp.npy')
    nf2=np.load('data/nf_multi.npy')
    x2=np.load('data/disp_multi.npy')
    
    #input6
    features1=np.concatenate((x1[:,-6:,:].reshape((-1,6*7)),nf1),axis=-1)
    features2=np.concatenate((x2[:,-6:,:].reshape((-1,6*7)),nf2),axis=-1)
    input6=np.concatenate((features1,features2));print('input6.shape,',input6.shape,);del features1,features2
    #label359
    features1=x1[:,:-6,:].reshape((-1,359*7))
    features2=x2[:,:-6,:].reshape((-1,359*7))
    label359=np.concatenate((features1,features2));print('label359.shape,',label359.shape,);del features1,features2
    #input 365
    features1=np.concatenate((x1.reshape((-1,365*7)),nf1),axis=1)
    features2=np.concatenate((x2.reshape((-1,365*7)),nf2),axis=1)
    features=np.concatenate((features1,features2));print('features.shape',features.shape);del nf1,features1,nf2,features2
    
    #x - split
    f_training, f_val, f_test = features[training_idx],features[val_idx], features[test_idx]
    input6_training, input6_val, input6_test = input6[training_idx],input6[val_idx], input6[test_idx]
    label359_training, label359_val, label359_test = label359[training_idx],label359[val_idx], label359[test_idx]
    print(f_training.shape,f_val.shape,f_test.shape)
    #x - save
    save_flag=False#True#
    if save_flag:
        data_path='data/data_mlp/multi/input365/'
        np.savez_compressed(data_path+'features_train',f_training)
        np.savez_compressed(data_path+'features_val',f_val)
        np.savez_compressed(data_path+'features_test',f_test)
        data_path='data/data_mlp/multi/input6/'
        np.savez_compressed(data_path+'features_train',input6_training)
        np.savez_compressed(data_path+'features_val',input6_val)
        np.savez_compressed(data_path+'features_test',input6_test)
        data_path='data/data_mlp/multi/label359/'
        np.savez_compressed(data_path+'label_train',label359_training)
        np.savez_compressed(data_path+'label_val',label359_val)
        np.savez_compressed(data_path+'label_test',label359_test)

#############################
##y - multi
if False:#True:#
    
    N=12121+75000
    #split idx
    np.random.seed(1)
    indices = np.random.permutation(N);print('indices',indices[0])
    n_training=int(N*0.6)
    n_val=int(N*0.2)
    n_test=N-n_training-n_val
    training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]
    
    y_area1=np.load('data/data_cablearea.npy');
    y_area2=np.load('data/data_cablearea_multi.npy');
    label_area=np.concatenate((y_area1,y_area2));print('label_area.shape',label_area.shape); del y_area1,y_area2
    ##y - split
    l_training, l_val, l_test = label_area[training_idx],label_area[val_idx], label_area[test_idx]
    print(l_training.shape,l_val.shape,l_test.shape)
    #multi task
    def getting_multi(label_data,sf,mode):
        bce_data=(label_data<0.999).astype('uint8')
        bce_safe=np.sum(bce_data,axis=1)==0
        bce_safe=bce_safe.astype('uint8')[:,np.newaxis];print(np.sum(bce_safe))
        bce_data=np.concatenate((bce_safe,bce_data),axis=1)
        if sf:
            np.savez_compressed('data/data_mlp/label_multitask_bce/c_label_'+mode,bce_data)
            #np.savez_compressed('data/data_mlp/label_multitask_bce/a_label_'+mode,a_data)
        return bce_data

    save_flag=False#True#
    bce_training=getting_multi(l_training,sf=save_flag,mode='train')
    bce_val=getting_multi(l_val,sf=save_flag,mode='val')
    bce_test=getting_multi(l_test,sf=save_flag,mode='test')
    
    save_flag=False#True#
    if save_flag:
        data_path='data/data_mlp/label/'
        np.savez_compressed(data_path+'label_train',l_training)
        np.savez_compressed(data_path+'label_val',l_val)
        np.savez_compressed(data_path+'label_test',l_test)


indices 8203
nf1.shape (12121, 7) x1.shape (12121, 365, 7)
input6.shape, (12121, 49)
label359.shape, (12121, 2513)
features.shape (12121, 2562)
(7276, 2562) (2422, 2562) (2423, 2562)
